In [159]:
from pathlib import Path
from pprint import pprint
import progressbar, csv, json, re

In [160]:
tweet_cache_dir = Path('./__cache__/tweets/')
user_cache_dir = Path('./__cache__/users/')

def get_tweet(_id):
    _json = None
    cache_file = tweet_cache_dir / _id
    if cache_file.is_file:
        with cache_file.open() as f:
            _json = json.load(fp=f)
    return(_json)

def expand_tags_tsv(file):
    all_captions = {}
    count_not_downloaded = 0
    _json = None
    
    if Path(file).is_file(): _ = [Path(file)]
    elif Path(file).is_dir(): _ = Path(file).glob(pattern="*.tsv")
    else:
        raise RuntimeError(f"Cannot interpret passed argument: {file}.")
    for file in _:
        with Path(file).open("r") as f:
            _len = len(f.readlines())
        with Path(file).open("r") as f:
            bar = progressbar.ProgressBar(max_value=_len).start()
            reader = csv.DictReader(f, delimiter='\t')
            for i, rows in enumerate(reader):
                bar.update(i)
                skip = False
                try:
                    id_int = int(rows['id_str'])
                    _json = get_tweet(rows['id_str'])
                except Exception as e:
                    # if "invalid literal" in str(e): print(f"Not a valid ID on line {i}.")
                    skip = True
                
                if not skip and _json is not None:
                    if _json['json_source'] == "Twitter":
                        # We have a JSON file and it comes from Twitter
                        
                        if not _json['retweeted']:
                            clean_text = _json['full_text']
                            clean_text = ' '.join(re.sub("(^RT )|(#[A-Za-z0-9_]+)|(@[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(\b(I|II|III|IV|V|VI|VII|VIII|IX|X|XI|XII|XIII|XIV|XV|XVI|XVII|XVIII|XIX|XX))"," ",clean_text).split())
                            clean_text = clean_text.lower()
                            if "burlesque" in clean_text:
                                clean_text_first_8_words = " ".join(clean_text.split(" ")[:8])
                                if clean_text_first_8_words not in all_captions: all_captions[clean_text_first_8_words] = 0
                                all_captions[clean_text_first_8_words] += 1
                    elif _json['json_source'] == "TAGS":
                        # We have a limited JSON file and it comes from TAGS (which also means it contains the error?)
                        if "63" in _json['error']: pass # User has been suspended
                        elif "179" in _json['error']: pass # Not authorized to see status.
                        elif "144" in _json['error']: pass # No status found with that ID.
                        elif "34" in _json['error']: pass # No such page exists
                        else: 
                            pprint(_json['error'])
                            print(tweet_cache_dir / str(id_int), "\n")
                        pass
                elif _json is None:
                    count_not_downloaded += 1
    if count_not_downloaded > 0: print(f"Warning: {count_not_downloaded} tweets could not be loaded (because they haven't been downloaded yet).")
    return(all_captions)
                    


In [165]:
all_captions = expand_tags_tsv('../../datasets/tags-tsv/burlesque/TAGS - burlesque 1 - Archive.tsv')

 99% (83991 of 84122) |################# | Elapsed Time: 0:01:06 ETA:   0:00:00

In [157]:
import operator
sorted_all_captions = sorted(all_captions.items(), key=operator.itemgetter(1), reverse=True)
for i, caption in enumerate(sorted_all_captions):
    if int(caption[1]) >= 2: pass
    else: del sorted_all_captions[i]
pprint(sorted_all_captions)

[('photography series captures male burlesque performers all around', 16),
 ('8 gennaio bianconiglio burlesque boylesque fe', 13),
 ('do count as maps here s one of', 6),
 ('male burlesque entertainers please wear pasties in solidarity', 4),
 ('all male burlesque show glitter bombs the world', 4),
 ('bella serata iniziamo il 2016 col botto mega', 4),
 ('sparkly beaded flat boylesque or burlesque pasties in', 3),
 ('naughty little cabaret chicago s only male burlesque', 3),
 ('don t miss the lovely and 24 other', 3),
 ('saturday january 23rd come see 25 burlesque and', 3),
 ('at 5 8pm wednesday feb 17 burlesque boylesque', 2),
 ('challenging gender beliefs of being a feminine heterosexual', 2),
 ('i vote for a burlesque and boylesque show', 2),
 ('when you lose your job and your wife', 2),
 ('variety of upcoming events at the encore entertainment', 2),
 ('burlesque and boylesque and drag queens oh my', 2),
 ('briefs fantastic all male burlesque show udderbelly festival', 2),
 ('saturday